# Pacific variability - PDO / IPO

original definition by _Mantua et al. (1997)_

> The leading EOF of monthly SST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)

---

0. create xr dataarrays of monthly Pacific data only  (from rect data for high res)
    1. North of 20 N
    2. North of Equator
    3. North of 38S

1. deseasonalize, detrend monthly SST data  (emphasis on consistency with other data analysis and not necessarily original definition)
    - HadISST:
        1. calculate monthly deviations (i.e. average difference) from annual mean, then remove this seasonal cycle
        2. two factor detrending with natural and anthropogenic forcing estimates at each grid point
    - CESM output:
        1. calculate monthly deviations (i.e. average difference) from annual mean, then remove this seasonal cycle
        2. remove quadratic trend at each grid point  (for different time segment)

2. EOF analysis of data

3. create annual index, lowpass filter index

4. analysis
    - spectra
    - regression patterns

In [ ]:
import os
import sys
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import cmocean
import cartopy
import warnings
import cartopy.crs as ccrs
import matplotlib
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file')
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from tqdm import tqdm
from paths import path_results, path_prace, file_HadISST
from filters import chebychev, lowpass
from regions import boolean_mask, global_ocean, gl_ocean_rect, gl_ocean_low, mask_box_in_region
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA, dll_dims_names
from xr_regression import xr_quadtrend
from ab_derivation_SST import DeriveSST as DS
from bd_analysis_indices import AnalyzeIndex as AI
from SST_index_generation import times_ctrl, times_lpd, times_had

# 1. data preparation

- deseasonalize monthly simulation output:
`DS().deseasonalize_monthly_data(run)`  (1min 52s for ctrl)
- detrend quadratically monthly simulation output:
`DS().detrend_monthly_data_pointwise(run)`  (ca 20min for ctrl, 4min 51s for lpd)
- two factor detrending of monthly HadISST data:
`DS().detrend_monthly_obs_two_factor()`  (4min 38sec)
- subselecting Pacific data
`DS().isolate_Pacific_SSTs(run, extent)`
- EOF analysis
`AI().Pacific_EOF_analysis(run, extent)`  (22min)

In [ ]:
DS().isolate_Pacific_SSTs(run='ctrl', extent='20N', time=times_ctrl[0])

In [ ]:
monthly = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_ctrl_{times_ctrl[0][0]}_{times_ctrl[0][1]}.nc')

In [ ]:
monthly[0,:,:].plot()

In [ ]:
area = xr.open_dataarray(f'{path_prace}/geometry/AREA_20N_ocn_rect.nc')

In [ ]:
monthly[0,:,:].plot()

In [ ]:
xx = monthly.where(area)

In [ ]:
xx[0,:,:].plot()

In [ ]:
monthly_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ctrl.nc')
monthly_lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_lpd.nc', decode_times=False)  # proper datetime
monthly_had  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_had.nc')

In [ ]:
yrly_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_rect_ctrl.nc')
yrly_lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_lpd.nc')
yrly_had  = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_had.nc')

In [ ]:
monthly_ds_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_ctrl_51_301.nc')
monthly_ds_lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_lpd_154_404.nc' , decode_times=False)
monthly_ds_had  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_had.nc' , decode_times=False)

In [ ]:
monthly_ds_ctrl[900:1200,100,100].plot()
monthly_ctrl[900:1200,100,100].plot()

# 2. deseasonalize and detrend

In [ ]:
monthly_ds_dt_had = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_tfdt_had.nc')
beta_anthro  = xr.open_dataset(f'{path_prace}/SST/SST_beta_anthro_MMM_monthly_had.nc')
beta_natural = xr.open_dataset(f'{path_prace}/SST/SST_beta_natural_MMM_monthly_had.nc')

f, ax = plt.subplots(2,2, figsize=(8,5))
beta_natural.forcing_natural.plot(ax=ax[0,0])
beta_anthro. forcing_anthro .plot(ax=ax[0,1])
beta_natural.beta_natural   .plot(ax=ax[1,0])
beta_anthro. beta_anthro    .plot(ax=ax[1,1])
ax[0,0].set_title('natural')
ax[0,1].set_title('anthropogenic')

# 3. EOF analysis

## subselect Pacific data

## Principal Components

In [ ]:
plt.figure(figsize=(8,5))
ax = plt.gca()
for i, extent in enumerate(['38S', 'Eq', '20N']):
    EOF_fns       = [[f'{path_prace}/SST/PMV_EOF_{extent}_ctrl_{time[0]}_{time[1]}.nc' for time in times_ctrl],
                     [f'{path_prace}/SST/PMV_EOF_{extent}_lpd_{time[0]}_{time[1]}.nc' for time in times_lpd], 
                     [f'{path_prace}/SST/PMV_EOF_{extent}_had.nc']]
    EOF_fns_yrly  = [[f'{path_prace}/SST/PMV_EOF_{extent}_raw_yrly_ctrl_{time[0]}_{time[1]}.nc' for time in times_ctrl],
                     [f'{path_prace}/SST/PMV_EOF_{extent}_raw_yrly_lpd_{time[0]}_{time[1]}.nc' for time in times_lpd], 
                     [f'{path_prace}/SST/PMV_EOF_{extent}_raw_yrly_had.nc']]
    ax.text(-20, 35*i+25, extent)
    ls = '-'
    for j, domain in enumerate(['ocn_rect', 'ocn_low', 'ocn_had']):
        (d, lat, lon) = dll_dims_names(['ocn_rect', 'ocn', 'ocn_had'][j])
        if j==0: continue
        c = f'C{j}'
        tf = [1,365,365][j]  # time factor
        to = [130,300,0][j]  # time offset
        EOF_fns_ = EOF_fns[j]
        EOF_fns_yrly_ = EOF_fns_yrly[j]
        for k, fn in enumerate(EOF_fns_):
            assert os.path.exists(fn)
#             assert os.path.exists(EOF_fns_yrly_[k])
            da = xr.open_dataset(fn, decode_times=False).pcs
            ax.plot(da.time/tf+to, lowpass(da,5*12)+3*k+35*i, c=c, ls=ls)
for i in range(3):
    ax.text([70, 300, 575][i], 105, ['OBS', 'HIGH', 'LOW'][i])
    plt.yticks([])
plt.ylim((-4,110))

In [ ]:
TPI_ctrl = xr.open_dataarray(f'{path_prace}/SST/TPI_ctrl.nc', decode_times=False)
TPI_lpd  = xr.open_dataarray(f'{path_prace}/SST/TPI_lpd_154_404.nc' , decode_times=False)
TPI_had  = xr.open_dataarray(f'{path_prace}/SST/TPI_had.nc' , decode_times=False)

In [ ]:
plt.figure(figsize=(8,5))
ax = plt.gca()
for i in range(4):
    ax.axhline(i, c='grey', lw=.5)
TPI_fns = [f'{path_prace}/SST/TPI_ctrl.nc',
           f'{path_prace}/SST/TPI_lpd.nc', 
           f'{path_prace}/SST/TPI_had.nc']
for i, fn in enumerate(TPI_fns):
    tf = [1,365,365][i]  # time factor
    to = [130,300,0][i]  # time offset
    da = xr.open_dataarray(fn, decode_times=False)
    ax.plot(da.time[:250]/365+to, 4*da[:250]+3, c=f'C{i}', lw=.3, alpha=.3)
    ax.plot(da.time[:250]/365+to, 4*lowpass(da[:250],13)+3, c=f'C{i}')
    
labels = []

for i, extent in enumerate(['38S', 'Eq', '20N']):
    EOF_fns       = [f'{path_prace}/SST/PMV_EOF_{extent}_ctrl_51_301.nc',
                     f'{path_prace}/SST/PMV_EOF_{extent}_lpd_154_404.nc',
                     f'{path_prace}/SST/PMV_EOF_{extent}_had.nc']
    for j, fn in enumerate(EOF_fns):
        (d, lat, lon) = dll_dims_names(['ocn_rect', 'ocn', 'ocn_had'][j])
        tf = [1,365,365][j]  # time factor
        to = [130,300,0][j]  # time offset
        da = xr.open_dataset(fn, decode_times=False).pcs
        ax.plot(da.time/tf+to, da+i, c=f'C{j}', lw=.3, alpha=.3)
        ax.plot(da.time/tf+to, lowpass(da,13*12)+i, c=f'C{j}')
    ax.text([70, 300, 575][i], 3.7, ['OBS', 'HIGH', 'LOW'][i])
    labels.append(f'PC(>{extent})')
labels.append('TPI (x4)')

plt.xlim((-30,750))
plt.ylim((-.8,4))
ax.set_yticks(range(4))
ax.set_yticklabels(labels)
plt.xlabel('time [years]')

In [ ]:
# before

## spectra

In [ ]:
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(8,8), sharex=True, sharey=True)

for i, extent in enumerate(['20N', 'Eq', '38S']):
    EOF_fns = [f'{path_prace}/SST/PMV_EOF_{extent}_had.nc',
               f'{path_prace}/SST/PMV_EOF_{extent}_ctrl_51_301.nc',
               f'{path_prace}/SST/PMV_EOF_{extent}_lpd_154_404.nc']
    ax[i,0].set_ylabel(extent)
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
#         if i>0 or j>0:  continue
        ax[i,j].axvline(1/5/12, c='grey', lw=.5)
        ax[i,j].set_xscale('log')
        ax[i,j].set_yscale('log')
        ts = xr.open_dataset(EOF_fns[j], decode_times=False).pcs.squeeze()
        if j>0:  ts = ts.assign_coords(time=ts.time.values/365)
            
#         f, Pxx_den = sp.signal.welch(ts.values)
#         ax[i,j].plot(f, Pxx_den, ls=':', c='C1')
#         f, Pxx_den = sp.signal.welch(lowpass(ts,5).values)
#         ax[i,j].plot(f, Pxx_den, ls=':', c='C1')
        
        (spec, freq, _) = ATS(ts).spectrum()
        ax[i,j].plot(freq, spec, c='C0')
        (spec, freq, _) = ATS(lowpass(ts,5*12)).spectrum()
        ax[i,j].plot(freq, spec, c='C0')
        AR_spectrum = ATS(ts).mc_ar1_spectrum(filter_type=None, filter_cutoff=None)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[0,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[2,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[3,:], ls='--', c='C3', lw=.5)
        
        AR_spectrum = ATS(ts).mc_ar1_spectrum(filter_type='lowpass', filter_cutoff=5*12)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[0,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[2,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[3,:], ls='--', c='C3', lw=.5)
        if i==0:
            ax[i,j].set_title(['HIST', 'HIGH', 'LOW'][j])
            ax[2,j].set_xlabel(r'frequency [month$^{-1}$]')
#         ax[i,j].set_xlim((2,120))
        ax[i,j].set_ylim((1e-2,2e2))

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(8,8), sharex=True, sharey=True)

for i, extent in enumerate(['20N', 'Eq', '38S']):
    EOF_fns = [f'{path_prace}/SST/PMV_EOF_{extent}_raw_yrly_had.nc',
               f'{path_prace}/SST/PMV_EOF_{extent}_raw_yrly_ctrl_51_301.nc',
               f'{path_prace}/SST/PMV_EOF_{extent}_raw_yrly_lpd_154_404.nc']
    ax[i,0].set_ylabel(extent)
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
#         if i>0 or j>0:  continue
        ax[i,j].axvline(1/5, c='grey', lw=.5)
        ax[i,j].set_xscale('log')
        ax[i,j].set_yscale('log')
        ts = xr.open_dataarray(EOF_fns[j], decode_times=False).squeeze()
        if j>0:  ts = ts.assign_coords(time=ts.time.values/365)
            
#         f, Pxx_den = sp.signal.welch(ts.values)
#         ax[i,j].plot(f, Pxx_den, ls=':', c='C1')
#         f, Pxx_den = sp.signal.welch(lowpass(ts,5).values)
#         ax[i,j].plot(f, Pxx_den, ls=':', c='C1')
        
        (spec, freq, _) = ATS(ts).spectrum()
        ax[i,j].plot(freq, spec, c='C0')
        (spec, freq, _) = ATS(lowpass(ts,5)).spectrum()
        ax[i,j].plot(freq, spec, c='C0')
        AR_spectrum = ATS(ts).mc_ar1_spectrum(filter_type=None, filter_cutoff=None)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[0,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[2,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[3,:], ls='--', c='C3', lw=.5)
        
        AR_spectrum = ATS(ts).mc_ar1_spectrum(filter_type='lowpass', filter_cutoff=5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[0,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[2,:], ls='--', c='C3', lw=.5)
        ax[i,j].plot(AR_spectrum[1,:], AR_spectrum[3,:], ls='--', c='C3', lw=.5)
        if i==0:
            ax[i,j].set_title(['HIST', 'HIGH', 'LOW'][j])
            ax[2,j].set_xlabel(r'frequency [year$^{-1}$]')
#         ax[i,j].set_xlim((2,120))
        ax[i,j].set_ylim((1e-2,5e1))

### autocorrelation maps

In [ ]:
from bc_analysis_fields import AnalyzeField as AF

In [ ]:
# for ctrl: create yrly files of segments
for i, run in enumerate(['ctrl', 'lpd']):
    for j, time in enumerate([times_ctrl, times_lpd][i]):
        print(time)
        fn     = f'{path_prace}/SST/SST_monthly_ds_dt_{run}_{time[0]}_{time[1]}.nc'
        fn_out = f'{path_prace}/SST/SST_yrly_from_monthly_ds_dt_{run}_{time[0]}_{time[1]}.nc'
        DS().generate_yrly_SST_ctrl_rect(fn=fn, fn_out=fn_out)

In [ ]:
da = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ctrl.nc')

In [ ]:
times_ctrl

## SST regression plots

In [ ]:
# %%time
# SST_rect_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_rect_ctrl.nc', decode_times=False)
# SST_rect_rcp  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_rect_rcp.nc' , decode_times=False)
# SST_rect_ds_dt_ctrl = lowpass(lowpass(notch(SST_rect_ctrl, 12), 12), 12) - SST_gm_rect_ds_ctrl[:-7]
# SST_rect_ds_dt_rcp  = lowpass(lowpass(notch(SST_rect_rcp , 12), 12), 12) - SST_gm_rect_ds_rcp[:-1]
# SST_rect_ds_dt_ctrl.to_netcdf(f'{path_prace}/SST/SST_monthly_rect_ds_dt_ctrl.nc')
# SST_rect_ds_dt_rcp .to_netcdf(f'{path_prace}/SST/SST_monthly_rect_ds_dt_rcp.nc' )

In [ ]:
SST_rect_ds_dt_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_rect_ds_dt_ctrl.nc', decode_times=False)
SST_rect_ds_dt_rcp  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_rect_ds_dt_rcp.nc' , decode_times=False)

In [ ]:
%%time
# 2:25 min
# ds_20N_ctrl = lag_linregress_3D(Pac_20N_ctrl.pcs[:-7,0], SST_rect_ds_dt_ctrl[24:-(24+7)], dof_corr=1./(12*13))
ds_38S_ctrl = lag_linregress_3D(Pac_38S_ctrl.pcs[:-7,0], SST_rect_ds_dt_ctrl[24:-(24+7)], dof_corr=1./(12*13))
# ds_20N_rcp  = lag_linregress_3D(-Pac_20N_rcp.pcs[:-7,0], SST_rect_ds_dt_rcp [24:-(24+7)], dof_corr=1./(12*13))
ds_38S_rcp  = lag_linregress_3D(Pac_38S_rcp .pcs[:-7,0], SST_rect_ds_dt_rcp [24:-(24+7)], dof_corr=1./(12*13))


In [ ]:
for ds in [ds_20N_ctrl, ds_38S_ctrl]:
    ds.attrs['first_year'] = 102
    ds.attrs['last_year']  = 297
for ds in [ds_20N_rcp, ds_38S_rcp]:
    ds.attrs['first_year'] = 2002
    ds.attrs['last_year']  = 2097

In [ ]:
ds_20N_ctrl

In [ ]:
regr_map(ds=ds_20N_ctrl, index='PDO', run='ctrl', fn=None)

In [ ]:
regr_map(ds=ds_38S_ctrl, index='IPO', run='ctrl', fn=None)

In [ ]:
regr_map(ds=ds_20N_rcp, index='PDO', run='rcp', fn=None)

In [ ]:
regr_map(ds=ds_38S_rcp, index='IPO', run='rcp', fn=None)

In [ ]:
cartopy.__version__